# Generating Pokemon Names

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/9/98/International_Pok%C3%A9mon_logo.svg/640px-International_Pok%C3%A9mon_logo.svg.png)

In this exercise we will train a recurrent neural network that can generate names of Pokemons. We can retrieve data on Pokemons from the [Pokemon API](https://pokeapi.co/).


First, we will fetch the list of Pokemon names from the pokeapi. The names need a bit of preprocessing and deduplication. We will also add a **"."** to the end of each name. This will be an indication of the end of the name once we have a model that can generate new Pokemon names for us. 

In [1]:
%tensorflow_version 2.x

import numpy as np
import requests
import tensorflow as tf

result = requests.get("https://pokeapi.co/api/v2/pokemon/?limit=1000").json()
names = list(set([result['results'][i]['name'].split("-")[0]+"." for i in range(result['count'])]))

names[:20]

TensorFlow 2.x selected.


['dunsparce.',
 'beheeyem.',
 'krabby.',
 'granbull.',
 'milotic.',
 'sneasel.',
 'hippopotas.',
 'oricorio.',
 'komala.',
 'gumshoos.',
 'bulbasaur.',
 'maractus.',
 'piloswine.',
 'rhyhorn.',
 'togetic.',
 'solosis.',
 'octillery.',
 'toxicroak.',
 'lotad.',
 'tornadus.']

We will generate the names character by character, so first we need to identify all the unique characters used in the Pokemon names.

In [0]:
chars = set()

chars.update("".join(names))
char_to_index = dict(zip(chars, range(len(chars))))
index_to_char = {i: c for c, i in char_to_index.items()}

The two dictionaries **char_to_index** and **index_to_char** will be useful for one-hot-encoding the data.

We will now create the training data, which will have dimensions (number_of_names, maximum_length_of_name, size_of_alphabet). We will create both a tensor of features **X** and a tensor of targets **Y**. 

In [0]:
max_name_length = len(max(names, key=len))
m = len(names)
char_dim = len(char_to_index)

X = np.zeros((m, max_name_length, char_dim))
Y = np.zeros((m, max_name_length, char_dim))

for i in range(m):
    name = list(names[i])
    for j in range(len(name)):
        X[i, j, char_to_index[name[j]]] = 1
        if j < len(name)-1:
            Y[i, j, char_to_index[name[j+1]]] = 1

The model will have a LSTM layer and a dense softmax layer on top. Per default the LSTM layer would only return the state at the end of each name, but since we want every character in the name to count we can specify this with **return_sequences=True**.

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import LambdaCallback

model = Sequential()
model.add(LSTM(128, input_shape=(max_name_length, char_dim), return_sequences=True))
model.add(Dense(char_dim, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

To be able to actually generate names with the model we will write a function that samples one character at a time until either a "." is sampled or we reach the maximum length of Pokemon names.

In [0]:
def generate_name(model):
    name = []
    x = np.zeros((1, max_name_length, char_dim))
    end = False
    i = 0
    
    while end==False:
        probs = list(model.predict(x)[0,i])
        probs = probs / np.sum(probs)
        index = np.random.choice(range(char_dim), p=probs)
        if i == max_name_length-2:
            character = '.'
            end = True
        else:
            character = index_to_char[index]
        name.append(character)
        x[0, i+1, index] = 1
        i += 1
        if character == '.':
            end = True
    
    print(''.join(name))

In order to follow how our model is developing we use a callback function and generate three random names for every 25 epochs. Hopefully, we should see that the names are becomming more realistic as the model is trained.

Remember: If you run the training loop again, the model will start off with the trained weights unless you recompile the model again.

In [6]:
def generate_name_loop(epoch, _):
  
  if epoch%25 == 0:
    print('Name generated after epoch %d:' % epoch)
    for i in range(3):
      generate_name(model)
    print()
      
name_generator = LambdaCallback(on_epoch_end = generate_name_loop)

model.fit(X, Y, batch_size=64, epochs=300, callbacks=[name_generator], verbose=0)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Name generated after epoch 0:
zsupktyypjk.
2nlybef.
ubsvtjeaxbh.

Name generated after epoch 25:
uugdtbn.
bowyula.
xegddeur.

Name generated after epoch 50:
.
urodel.
ochedore.

Name generated after epoch 75:
eelattril.
ollitrone.
.

Name generated after epoch 100:
panaurr.
rongra.
wingoru.

Name generated after epoch 125:
mabraa.
indien.
ogufbunf.

Name generated after epoch 150:
agneogst.
uctron.
klitk.

Name generated after epoch 175:
ixer.
oman.
nichoon.

Name generated after epoch 200:
licoun.
actuffe.
ascung.

Name generated after epoch 225:
lidocio.
ochoynn.
antige.

Name generated after epoch 250:
udbray.
rofelta.
olitraka.

Name generated after epoch 275:
lickily.
irlion.
esceviar.



Do you think that the model is good?

Can you tell the generated Pokemon names from real ones?

What else do you think can be successfully generated with RNNs?